In [ ]:
%run fungsi_save.ipynb

In [ ]:
X = pd.read_csv('data_training.csv', sep=',',header=None) #read training data file
X = X.values #convert training data to array
total_features = np.size(X, axis=1)-2 #number of features
print(total_features)
a = [] #index fitur yang gak kepake
for i in range(len(X)):
    if X[i,29] == 2:
        a.append(i)
X = np.delete(X, a ,axis=0) #data non limfo dihapus
y = X[:,total_features] #output aja limfo
y = np.int_(y) #change the values to intiger
X = X[:,:total_features] #all features 

<h5>The code below refers to the example of features selection in PySwarms documentation example</h5>

In [ ]:
from sklearn import linear_model

# Create an instance of the classifier
classifier = linear_model.LogisticRegression()

# Define objective function
def f_per_particle(m, alpha):
    global total_features
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X
    else:
        X_subset = X[:,m==1]
    # Perform classification and store performance in P
    classifier.fit(X_subset, y)
    P = (classifier.predict(X_subset) == y).mean()
    # Compute for the objective function
    j = (alpha * (1.0 - P)
        + (1.0 - alpha) * (1 - (X_subset.shape[1] / total_features)))
    return j

In [ ]:
def f(x, alpha=0.88):
    n_particles = x.shape[0]
    j = [f_per_particle(x[i], alpha) for i in range(n_particles)]
    return np.array(j)

In [ ]:
# Initialize swarm, arbitrary
options = {'c1': 0.5, 'c2': 0.5, 'w':0.9, 'k': 30, 'p':2}

# Call instance of PSO
dimensions = total_features # dimensions should be the number of features

optimizer = ps.discrete.BinaryPSO(n_particles=30, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=25)

In [ ]:
# Create two instances of LogisticRegression
classifier = linear_model.LogisticRegression()

# Get the selected features from the final positions
X_selected_features = X[:,pos==1]  # subset
X_non_selected_features = X[:,pos==0]
# Perform classification and store performance in P
classifier.fit(X_selected_features, y)
print(np.size(X_selected_features,1))
# Compute performance
subset_performance = (classifier.predict(X_selected_features) == y).mean()

print('Subset performance: %.3f' % (subset_performance))

In [ ]:
a = []
for i in range(len(optimizer.cost_history)):
    a.append(i)
b = optimizer.cost_history
plt.plot(a,b)
plt.show()

In [ ]:
pos = [i for i in pos]
pos

In [ ]:
save_pos_step1('fix2(balanced)\step1\seleksi_fitur\hasil_step1.csv', (pos,optimizer.cost_history,subset_performance))